# A chatbot based on seq2seq model with attention

This chatbot follows the encoder-decoder approach, using [Bahdanau attention](https://arxiv.org/pdf/1409.0473.pdf).
* A GRU encoder maps a tokenized English sentence to a number of hidden states
* A GRU decoder then uses its own state and all encoder states to compute a context vector.
* During training, (at each time step) the decoder takes the context vector and the target output to predict an output sequence; during testing, the decoder takes the context vector and the previous prediction to compute the current prediction.

The inference experiments showed reasonably good results.

In [ ]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tqdm import tqdm
from sklearn.utils import shuffle

CONV_TSV_PATH = 'conv_pair.tsv'
TOKENIZER_PATH = 'tokenizer.pkl'

Prepare the dataset
I use [Cornell Movie Dialogs](http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html) as the training corpus.  
Prepation of the dataset uses some code from this [tensorflow official blog article](https://colab.research.google.com/github/tensorflow/examples/blob/master/community/en/transformer_chatbot.ipynb).  
But I used more sophisticated text cleaning heuristics. For example, the `html` marker `<u>...</u>` appears frequently in the raw text and should be removed.

In [ ]:
path_to_zip = tf.keras.utils.get_file(
    'cornell_movie_dialogs.zip',
    origin='http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip',
    extract=True)

path_to_dataset = os.path.join(os.path.dirname(path_to_zip), 
                               'cornell movie-dialogs corpus')
path_to_movie_lines = os.path.join(path_to_dataset, 'movie_lines.txt')
path_to_movie_conversations = os.path.join(path_to_dataset, 'movie_conversations.txt')

9920512/9916637 [==============================] - 3s 0us/step


In [ ]:
def text_clean(s):
    s = s.lower()
    s = re.sub(r'<u>|</u>', '', s)
    s = re.sub(r'[/(){}\[\]\|@]', ' ', s)
    s = re.sub(r'[^0-9a-z ,.!?]', ' ', s)
    s = re.sub(r'([,.!?])', r' \1 ', s)
    s = re.sub(r'\s+', ' ', s)
    return s.strip()

def load_conversations(MAX_SAMPLES=20000):
    id2line = {}
    with open(path_to_movie_lines, errors='ignore') as fin:
        for line in fin:
            parts = line.strip().split(' +++$+++ ')
            if len(parts) < 5: parts.append(' ')
            id2line[parts[0]] = parts[4]

    ques, ans = [], []
    with open(path_to_movie_conversations) as fin:
        for line in fin:
            parts = line.replace('\n', '').split(' +++$+++ ')
            conversation = [line[1:-1] for line in parts[3][1:-1].split(', ')]

            for i in range(len(conversation) - 1):
                q_text = text_clean(id2line[conversation[i]])
                a_text = text_clean(id2line[conversation[i + 1]])
                if q_text and a_text:
                    ques.append(q_text)
                    ans.append(a_text)
                if len(ques) >= MAX_SAMPLES: break
            
            if len(ques) >= MAX_SAMPLES: break
  
    return pd.DataFrame(list(zip(ques, ans)), 
                        columns=['ques', 'ans'])

## Tokenization
I saved the lists of questions and answers in a `tsv` file for later use.  
I used **tensorflow's** `SubwordTextEncoder` tokenizer which has a byte-level fallback. The tokenizer is saved for later use too.

In [ ]:
# create the dataframe
df = load_conversations()
df.to_csv(CONV_TSV_PATH, sep='\t', index=False)

# build vocabulary & tokenzier from examples
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    df['ques'] + df['ans'], target_vocab_size=2**13)
START_TOK = [tokenizer.vocab_size]
END_TOK = [tokenizer.vocab_size + 1]

pickle.dump((tokenizer, START_TOK, END_TOK), 
            open(TOKENIZER_PATH, 'wb'))

In [ ]:
# load the dataset & the tokenizer
df = pd.read_csv(CONV_TSV_PATH, sep='\t')
tokenizer, START_TOK, END_TOK = pickle.load(open(TOKENIZER_PATH, 'rb'))
VOCAB_SIZE = tokenizer.vocab_size + 2

In [ ]:
def texts2ids(texts, tokenizer, max_len=40, doclean=False):
    '''list of texts -> list of token ID arrays'''
    tokenized = []
    for t in texts:
        if doclean: t = text_clean(t)
        sent = START_TOK + tokenizer.encode(t) + END_TOK
        if len(sent) > max_len:
            sent = sent[:max_len - 1] + END_TOK
        tokenized.append(sent)
        
    return tf.keras.preprocessing.sequence.pad_sequences(
        tokenized, maxlen=max_len, padding='post')


I use only 20000 question-answer pairs. Now they are converted to token ID lists.

In [ ]:
# prepare tokenized questions and answers
SAMPLE_SIZE = 20000
X, y = shuffle(df['ques'], df['ans'], 
               random_state=0,
               n_samples=SAMPLE_SIZE)
X, y = texts2ids(X, tokenizer), texts2ids(y, tokenizer)


This is what the dataset, when de-tokenized, looks like

In [ ]:
for X1, y1 in zip(X[:5], y[:5]):
    print('Q:', tokenizer.decode([t for t in X1 if t < tokenizer.vocab_size]))
    print('A:', tokenizer.decode([t for t in y1 if t < tokenizer.vocab_size]))

Q: right . farewell . . . for the last time . . . may the gods prevent . . .
A: no , don t say anything else !
Q: the end of the recordare statuens in parte dextra .
A: so now the confutatis . confutatis maledictis . when the wicked are confounded . flammis acribus addictis . how would you translate 
Q: remember the guy who cheated at the table ?
A: you don t like cheats , do you .
Q: you can t have sven s father sitting next to sven . they ll argue the whole time .
A: that s true . you d better sit there . you there , and ornulf there .
Q: the bedroom .
A: there is only one bed .


## Encoder-decoder with attention
In defining the model architecture, I use code from this [tensorflow tutorial](https://www.tensorflow.org/tutorials/text/nmt_with_attention)

In [ ]:
# architecture

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, emb_size, enc_units,
                       batch_size, dropout):
        super().__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, emb_size)
        self.gru = tf.keras.layers.GRU(
            self.enc_units,
            return_sequences=True,
            return_state=True,
            dropout=dropout,
            recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state
    
    def initial_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))


class BAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)

        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        weights = tf.nn.softmax(score, axis=1)
        context = weights * values
        context = tf.reduce_sum(context, axis=1)

        return context, weights


class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, emb_size, dec_units,
                       batch_size, dropout):
        super().__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, emb_size)
        self.gru = tf.keras.layers.GRU(
            self.dec_units,
            return_sequences=True,
            return_state=True,
            dropout=dropout,
            recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = BAttention(dec_units)

    def call(self, x, hidden, enc_output):
        x = self.embedding(x)

        context, weights = self.attention(hidden, enc_output)
        x = tf.concat([tf.expand_dims(context, 1), x], axis=-1)
        
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)

        return x, state, weights

## Batch generator, hyper-parameters, optimizer, ...  
I use some code from the previous tensorflow tutorial

In [ ]:
BATCH_SIZE = 64
STEPS_PER_EPOCH = len(X) // BATCH_SIZE

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': X,
        'dec_inputs': y[:, :-1]
    },
    {
        'targets': y[:, 1:]
    }
))

dataset = dataset.cache()
dataset = dataset.batch(BATCH_SIZE)

In [ ]:
EMBEDDING_SIZE = 256
HIDDEN_UNITS = 256
DROPOUT_RT = 0.5

In [ ]:
# create encoder, decoder
encoder = Encoder(VOCAB_SIZE, EMBEDDING_SIZE, HIDDEN_UNITS, 
                  BATCH_SIZE, DROPOUT_RT)
decoder = Decoder(VOCAB_SIZE, EMBEDDING_SIZE, HIDDEN_UNITS, 
                  BATCH_SIZE, DROPOUT_RT)

# Optimizer and the loss
optimizer = tf.keras.optimizers.Adam()
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


def loss_func(real, pred):
    loss = loss_obj(real, pred)
    mask = tf.cast(tf.math.not_equal(real, 0), dtype=loss.dtype)
    return tf.reduce_mean(loss * mask)


@tf.function
def train_step(inputs, dec_inputs, targets, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inputs, enc_hidden)
        dec_hidden = enc_hidden

        for i in range(targets.shape[1]):
            pred, dec_hidden, _ = decoder(
                tf.expand_dims(dec_inputs[:, i], 1),
                dec_hidden,
                enc_output)
            
            loss += loss_func(targets[:, i], pred)

    batch_loss = loss / int(targets.shape[1])
    trainable = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, trainable)
    optimizer.apply_gradients(zip(gradients, trainable))

    return batch_loss

## Details of the training experiment
I trained the model for 40 epochs in two passes.  
The training loss of the 0th epoch was **2.12**. The figure dropped to **0.77** at the 39th epochs.  
The logging of training is a bit long ...  
The weights of the models are saved for later use.

In [ ]:
EPOCHS = 20

for epoch in range(EPOCHS):
    enc_hidden = encoder.initial_hidden_state()
    total_loss = 0

    for n, batch in tqdm(enumerate(dataset.take(STEPS_PER_EPOCH))):
        batch_loss = train_step(
            batch[0]['inputs'], 
            batch[0]['dec_inputs'], 
            batch[1]['targets'],
            enc_hidden)

        total_loss += batch_loss
        if n % 50 == 0:
            print(' Epoch %s Batch %s Loss %.4f' % (
                epoch, n, batch_loss.numpy()))
    
    print(' Epoch %s Loss %.4f\n' % (
        epoch, total_loss / STEPS_PER_EPOCH))


encoder.save_weights('enc_weights.h5')
decoder.save_weights('dec_weights.h5')

2it [00:31, 21.94s/it]

 Epoch 0 Batch 0 Loss 3.2251


52it [00:37,  7.99it/s]

 Epoch 0 Batch 50 Loss 2.2151


102it [00:43,  7.92it/s]

...

 Epoch 19 Batch 250 Loss 1.1285


302it [00:38,  7.88it/s]

 Epoch 19 Batch 300 Loss 1.2011


312it [00:39,  7.85it/s]


 Epoch 19 Loss 1.1917



In [ ]:
# same as above, another 20 epochs

2it [00:00,  6.24it/s]

 Epoch 20 Batch 0 Loss 1.1127


52it [00:06,  8.01it/s]

 Epoch 20 Batch 50 Loss 1.1451


102it [00:12,  8.19it/s]

...

 Epoch 39 Batch 250 Loss 0.7483


302it [00:37,  8.08it/s]

 Epoch 39 Batch 300 Loss 0.7880


312it [00:38,  8.13it/s]


 Epoch 39 Loss 0.7753



## Inference Experiments
We need an encoder and a decoder for inference. They use the weights saved above.

In [ ]:
# load the trained model for inference

def load_models(vocab_size, emb_size, hidden_units):
    encoder = Encoder(vocab_size, emb_size, hidden_units, 
                    1, 0)
    decoder = Decoder(VOCAB_SIZE, EMBEDDING_SIZE, HIDDEN_UNITS, 
                    1, 0)
    enc_hidden = encoder.initial_hidden_state()
    place_holder = tf.convert_to_tensor([[1]])
    enc_output, enc_hidden = encoder(place_holder, enc_hidden, training=False)
    decoder(place_holder, enc_hidden, enc_output, training=False)
    encoder.load_weights('enc_weights.h5')
    decoder.load_weights('dec_weights.h5')
    return encoder, decoder

inf_enc, inf_dec = load_models(VOCAB_SIZE, EMBEDDING_SIZE, HIDDEN_UNITS)

### Repetition
It turns out that the decoder doesn't always produce an `END-OF-SEQ` token, but produces repetition.
Thus the inference function **falls back** to [Floyd's algorithm](https://en.wikipedia.org/wiki/Cycle_detection) (which is of $O(n)$ time complexity) to manually detect cycles in the output sequence only when there is no `END-OF-SEQ`.

In [ ]:
def floyd(L):
    '''Floyd's cycle-finding algorithm'''

    for i in range(1, len(L)):
        if 2 * i < len(L) and L[i] == L[2 * i]:
            v = i; break
    else:
        return slice(0, len(L))

    for i in range(len(L)):
        if L[i] == L[v + i]:
            mu = i; break
    
    for i in range(1, len(L) - mu):
        if L[mu] == L[mu + i]:
            lam = i; break

    return slice(mu, mu + lam)


def infer(sent, max_len=40, doclean=True, decycle=True):   
    inputs = texts2ids([sent], tokenizer, doclean=doclean)
    enc_hidden = inf_enc.initial_hidden_state()
    enc_output, enc_hidden = inf_enc(inputs, enc_hidden, training=False)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims(START_TOK, 1)
    result = []
    for i in range(max_len):
        pred, dec_hidden, _ = inf_dec(
            dec_input, dec_hidden, enc_output, training=False)
        pred_id = tf.argmax(pred, axis=1).numpy()
        
        if pred_id[0] == END_TOK[0]:
            break
        else:
            result.append(pred_id)
            dec_input = tf.expand_dims(pred_id, 1)
    
    for sent in np.stack(result, axis=1):
        if decycle: sent = sent.tolist()[floyd(sent)]
        sent = [t for t in sent if t < tokenizer.vocab_size]
        return tokenizer.decode(sent)

In [ ]:
test_sents = ['Hi',
              'How are you?',
              "What's your name?",
              'Tell me about yourself',
              'Do you love me?',
              "What's the meaning of life?",
              'How is the weather today?',
              "Let's have a dinner",
              'Are you a bot?',
              'Are you a robot?',
              'Why not?',
              'Harry Potter?']

for s in test_sents:
    print('Q: %s\nA: %s\n' % (s, infer(s)))

Q: Hi
A: hi . what are you talkin about ?

Q: How are you?
A: i don t know what you re a little taste of you to get excited . 

Q: What's your name?
A: don t know . you 

Q: Tell me about yourself
A: i ve been trails .

Q: Do you love me?
A: no . . . 

Q: What's the meaning of life?
A: what the hell are you ?

Q: How is the weather today?
A: you re a little rules these men 

Q: Let's have a dinner
A: it s not anymore . 

Q: Are you a bot?
A: he s not going . 

Q: Are you a robot?
A: yeah . it was in the world .

Q: Why not?
A: i m not a little acting . 

Q: Harry Potter?
A: that ain t that he was in the world .



### Quality
* The responses are generally meaningful grammatical, with a correct use of punctuations.
* "_are you a bot?_" got a weird answer, but "_are you a robot_" got a better one.
* The chatbot is sometimes rude ("_what's the meaning of life_" -> "_what the hell are you?_")
* Here's what happens if we __don't__ manually detect cycles.

In [ ]:
for s in (test_sents[2], test_sents[4]):
    print('Q: %s\nA: %s\n' % (s, infer(s, decycle=False)))

Q: What's your name?
A: i don t know . you don t know . you don t know . you don t know . you don t know . you don t know . you don t know . you don t know . 

Q: Do you love me?
A: no . . . no . . . no . . . no . . . no . . . no . . . no . . . no . . . no . . . no . . . no . . . no . . . no . . . no . . . no . . . no . . . no . . . no . . . no . . . no . . . 



### Comparison with baseline
Chatterbot is used as the baseline: some of its answers are not as good as ours!

In [ ]:
def get_baseline():
    from chatterbot import ChatBot
    bot = ChatBot('Bot', trainer='chatterbot.trainers.ChatterBotCorpusTrainer')
    bot.train("chatterbot.corpus.english")
    
    return lambda x: bot.get_response(x)

baseline_bot = get_baseline()

In [ ]:
for s in test_sents:
    print('Q: %s\nA: %s\n' % (s, baseline_bot(s)))

Q: Hi
A: How are you doing?

Q: How are you?
A: Who? Who is but a form following the function of what

Q: What's your name?
A: I am still young

Q: Tell me about yourself
A: What do you want to know?

Q: Do you love me?
A: I could get mad.

Q: What's the meaning of life?
A: is a literary reference to the illuminatus.

Q: How is the weather today?
A: Maradona is great. Sinsemillia is even better.

Q: Let's have a dinner
A: What is your question?

Q: Are you a bot?
A: Yes I am.

Q: Are you a robot?
A: Yes I am.

Q: Why not?
A: i support the 2nd amendment.

Q: Harry Potter?
A: I am a programmer

